In [1]:
# Installing dependencies
import numpy as np
import pandas as pd
from yfQuery import datareader

In [2]:
class DataLoader:
    '''
        Class for loading past stocks prices
        symbol: symbol can be either a single symbol or 
                a list of symbol
        start:  start date
        end:    end date
    '''
    def __init__(self, symbol, start, end):
        # Preload
        self.data = datareader(symbol, start, end)
    
    def get(self, start, end):
        # Return a period of the data
        return self.data.iloc[start: end]

In [3]:
d = DataLoader('AAPL', '2018-01-01', '2019-12-31')

In [4]:
prices = d.data.iloc[0]

In [5]:
def price_df(price, period=390):
    # Create datetime index by the minutes
    # Start time : 9:30AM
    # 390 minuntes per trading day
    ts = price.name
    ts = ts.replace(hour=9, minute=31)
    index = pd.date_range(ts, periods=period, freq="1min")

    # Stacking all price array
    array = np.stack([np.zeros(period) + price['Open'],
                      np.zeros(period) + price['High'],
                      np.zeros(period) + price['Low'],
                      np.zeros(period) + price['Close']
                      ]).T
    return pd.DataFrame(array, index=index, columns=['Open', 'High', 'Low', 'Close'])

In [29]:
period = 390 * 4
array = np.zeros(period)
array[[0, -1]] = [prices['Open'], prices['Close']]
# High and Low indice
close_to_low = prices['Close'] / prices['Low'] - 1
open_to_low = prices['Open'] / prices['Low'] - 1
swap = True if open_to_low < close_to_low else False
reduce = np.random.choice([int(period * .40), int(period * .60)])

In [30]:
if swap:
    low_i = np.random.choice(np.arange(1, reduce))
    high_i = np.random.choice(np.arange(low_i, period - 1))
else:
    high_i = np.random.choice(np.arange(1, reduce))
    low_i = np.random.choice(np.arange(high_i, period - 1))